In [494]:
import pandas as pd
from finta import TA
from pathlib import Path
import random

df_demo = pd.read_csv(Path('raw_data/SPY_data.csv'))
df_demo_demo = '../raw_data/SPY_data.csv'


In [495]:
def create_df(df_ohlcv):
    # create a base dataframe from an imported csv file
    df = pd.read_csv(Path(f'raw_data/{df_ohlcv}'))
    df = df.drop(columns='Unnamed: 0')
    df['date'] = pd.to_datetime(df['date']).dt.date
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    return df 

In [496]:
def choose_indicator(filepath, indicator):
    #Pick an indicator to addd to base dataframe
    indicator = indicator.upper() 

    df = create_df(filepath)
    if indicator == 'ATR':
        df['ATR'] = TA.ATR(df)
        
    elif indicator == 'BBANDS':
        df[['BBANDS Upper','Moving Average', 'BBands Lower']] = TA.BBANDS(df)

    elif indicator == 'VBM':
        df['VBM'] = TA.VBM(df)

    elif indicator == 'KAMA':
        df['KAMA'] = TA.KAMA(df)

    elif indicator == 'DYMI':
        df['DYMI'] = TA.DYMI(df)

    elif indicator == 'KC':
        df[['KC Upper', 'KC Lower']] = TA.KC(df)

    elif indicator == 'APZ':
        df[['APZ Upper', 'APZ Lower']] = TA.APZ(df)
       
    elif indicator == 'MI':
        df['MI'] = TA.MI(df)

    else:
        print("Indicator not a volatility indicator")
         

    return df

In [497]:

def custom_pair_indicators(filepath, indicator_1, indicator_2):
    # pair any two indicators into a single dataframe
    # need to create two instances of choose indicator 
    indicator_1 = indicator_1.upper()
    indicator_2 = indicator_2.upper()
    
    df_1 = choose_indicator(filepath, indicator_1)
    df_2 = choose_indicator(filepath, indicator_2)
    
    df = df_1.merge(df_2, how='inner', right_index=True, left_index=True)
    df = df.drop(columns=['open_y', 'high_y','low_y','close_y', 'volume_y','trade_count_y','vwap_y'])
    df = df.rename(columns={'open_x':'open', 'high_x':'high', 'low_x':'low', 'close_x':'close', 'volume_x':'volume', 'trade_count_x':'trade_count', 'vwap_x':'vwap'})
    return df

In [498]:
def drop_any_column(df, *drop_list):
    # drop any column you like
    for drop_me in drop_list:
        df = df.drop(columns=drop_me)
    return df


In [499]:
def mix_pair_indicators(df_1, df_2):
    # merge two different combinatoins of indicators into one frame
    df = df_1.merge(df_2, how='inner', right_index=True, left_index=True)
    df = df.drop(columns=['open_y', 'high_y','low_y','close_y', 'volume_y','trade_count_y','vwap_y'])
    df = df.rename(columns={'open_x':'open', 'high_x':'high', 'low_x':'low', 'close_x':'close', 'volume_x':'volume', 'trade_count_x':'trade_count', 'vwap_x':'vwap'})
    return df


In [500]:
def apply_rolling_window(df, period,*columns):
    # apply rolling window to any column
    for col in columns:
        df[f'{col}'] = df[f'{col}'].rolling(window=period).mean()
    return df


In [531]:
example = custom_pair_indicators(df_demo_demo, "VBM", 'MI')
len(example.index)

1594

In [502]:
def get_column_range(df,period,*columns):
        # get the rolling range of a column
        for col in columns:
                high = df[f'{col}'].rolling(window=period).max()
                low = df[f'{col}'].rolling(window=period).min()
                df[f'{col} {period} period rolling range'] = high -low
        return df


In [503]:
# create a function that takes indicators and drop columns as arguemt
# then generates random strings to get indicators merged on single dataframe
def generate_random_df(filepath, rolling_window=False, drop_basic_column=False, column_range=False):
    indicator_list = ['ATR', 'DYMI', 'APZ','KC','VBM','BBANDS','MI', 'KAMA']
    all_column_list = ['ATR', 'DYMI', 'APZ','KC','VBM','BBANDS','MI', 'KAMA', 'close','open','high','low']
    base_columns = ['close','open','high','low','trade_count','vwap']
    indicator_1 = random.choice(indicator_list)
    indicator_2 = random.choice(indicator_list)
    if indicator_2 != indicator_1:
        df = custom_pair_indicators(filepath, indicator_1, indicator_2)
        if rolling_window == True:
            random_window = random.choice(range(2,10))
            column_rolling = random.choice(base_columns)          
            # not accepting my columns perameter
            df = apply_rolling_window(df,random_window,column_rolling)
        if drop_basic_column == True:
            drop_me = random.choice(base_columns)
            df = drop_any_column(df, drop_me)
        if column_range == True:
            range_period = random.choice(range(2,10))
            df = get_column_range(df, range_period, indicator_1)
            
        # indicators with mulitple columns not passing though    

    return df 
        

In [504]:
df_1 = generate_random_df(df_demo_demo, rolling_window=True,drop_basic_column=True)
df_1.tail()

,open,high,low,close,volume,vwap,ATR,DYMI
date,,,,,,,,
2022-03-24,445.82,449.856667,444.760,450.48,61411569,447.609014,8.861557,61.665813
2022-03-25,451.09,450.656667,448.430,452.69,77199328,451.047221,8.262143,73.744729
2022-03-28,452.04,453.130000,450.060,455.91,68356914,453.294974,7.816429,77.127858
2022-03-29,461.75,456.986667,457.180,461.55,86315479,459.632252,7.309286,76.543857
2022-03-30,460.29,459.725000,456.465,458.70,79547916,459.159522,7.180357,81.023608


# create an explanation summary function of each time the function is ran
# then upload each epoch to a dataframe
# two moving exponential moving averages
# look into the angle of a moving average

create a function that combine all indicators into one df change the moving average rolling period randomly to random columns

things to do:

-take in the filepath and create a csv


-combine all of the indicators using for loop and the indicator function


-create a randomness to apply different rolling averages to different columns


-create a df that records all of the inputs to each individual dataframe



In [ ]:
def random_rolling_columns(filepath):
    df = create_df(filepath)
    indicators = ['ATR', 'DYMI','VBM','MI', 'KAMA']
    for signal in indicators:
        column = choose_indicator(filepath,signal)
        df[signal] = column[signal]
        

            

        
    return df

random_rolling_columns(df_demo_demo)
    

In [ ]:
def random_rolling_columns(filepath):
    df = create_df(filepath)
    indicators = ['ATR', 'DYMI','VBM','MI', 'KAMA', 'BBANDS', 'APZ','KC']
    for signal in indicators:
        column = choose_indicator(filepath,signal)
        if signal == "KC" or 'APZ':
            df[f'{signal} Upper'] = column.iloc[:,7:7]
            df[f'{signal} Lower'] = column.iloc[:,8:8]
        # elif

        else:
         df[signal] = column[signal]
        

        
    return df

random_rolling_columns(df_demo_demo)

In [516]:
import import_ipynb
import ying
import Laramie
import utils_laramie

In [517]:
def calc_weekly_range_MODIFIED(df):
    """ Must run "get_day_names" first"""
    df = utils_laramie.get_day_names(df)
    week_high = 0  
    week_low = 9999999
    for index, row in df.iterrows():
        if df.loc[index, 'DayOfWeek'] == 'Monday':
            week_high = df.loc[index,row]
            week_low = df.loc[index,row]
        else: 
            if df.loc[index,row] > week_high:
                week_high=df.loc[index,row]
            if df.loc[index,row] < week_low:
                week_low=df.loc[index,row]
            if df.loc[index,'DayOfWeek'] == 'Friday':
                df.loc[index,'weekly_range'] = week_high - week_low
    df = df.fillna(0)
    return df.drop(columns="DayOfWeek")

# create functions where we can add indicators into dataframe with the rest of the data 
-maybe create a weekly averaeg for each indicator and add that to the df_1

-create more csvs with indicators

In [527]:
def configure_indicators(df_1):
    df1_indicators = df_1.iloc[:,6:]
    # df1_indicators = calc_weekly_range_MODIFIED(df1_indicators)
    df1_indicators = Laramie.grp_y_wk_d(df1_indicators)
    df1_indicators = Laramie.drop_off_weeks(df1_indicators)
    # df = pd.concat([df_1,df1_indicators],axis=0)
    return df1_indicators
me = configure_indicators(df_1)
me

,ATR,DYMI
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
1050,10.399414,41.516734
1051,10.251557,41.338068
1052,9.734414,49.047465
1053,9.584414,77.052721


In [508]:
def standard_df():
    df_1 = (Laramie.get_df(Laramie.get_all_raw_data()))
    df_1 = Laramie.calc_weekly_range(df_1)
    df_1 = Laramie.grp_y_wk_d(df_1)
    df_1 = Laramie.drop_off_weeks(df_1)
    df_1 = df_1.drop(columns={'VIX_open','VIX_high','VIX_low','VIX_close'})
    return df_1

standard_df()

,SPY_open,SPY_high,SPY_low,SPY_close,SPY_volume,SPY_trade_count,SPY_vwap,TR,weekly_range
0,209.20,209.7295,207.20,208.27,102027111.0,374705.0,208.276128,2.5295,0.0000
1,206.48,208.2890,205.78,206.99,103372367.0,387782.0,206.966276,2.5090,0.0000
2,206.20,208.6800,204.18,205.33,162401537.0,586210.0,206.034646,4.5000,0.0000
3,205.44,207.4300,205.14,205.86,116128858.0,404992.0,206.102975,2.2900,0.0000
4,203.38,204.1400,201.51,201.88,211173305.0,669924.0,203.150102,4.3500,8.2195
...,...,...,...,...,...,...,...,...,...
1050,420.98,424.5500,415.79,417.00,91251505.0,858504.0,419.220077,8.7600,0.0000
1051,419.66,426.8400,418.42,426.17,104219651.0,920659.0,422.752423,9.8400,0.0000
1052,429.94,435.6800,424.80,435.55,138130298.0,1344164.0,431.339744,10.8800,0.0000
1053,433.70,441.0700,433.19,441.07,100157174.0,784018.0,437.706981,7.8800,0.0000
